In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import os

In [5]:
pip install tensorflow

  Using cached tensorflow-2.3.0-cp38-cp38-win_amd64.whl (342.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.7.0-py3-none-any.whl (779 kB)
  Using cached Markdown-3.2.2-py3-none-any.whl (88 kB)
  Using cached cachetools-4.1.1-py3-none-any.whl (10 kB)

  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.6-py3-none-any.whl (47 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Created wheel for termcolor:

In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
data = pd.read_csv('c:/Users/Ruchi/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
from sklearn import preprocessing
labels = preprocessing.LabelEncoder()
data['sentiment'] = labels.fit_transform(data['sentiment'])
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
import warnings
warnings.simplefilter('ignore')

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

vect_texts = vectorizer.fit_transform(list(data['review']))

all_ngrams = vectorizer.get_feature_names()
num_ngrams = min(50, len(all_ngrams))
all_counts = vect_texts.sum(axis=0).tolist()[0]

all_ngrams, all_counts = zip(*[(n, c) for c, n in sorted(zip(all_counts, all_ngrams), reverse=True)])
ngrams = all_ngrams[:num_ngrams]
counts = all_counts[:num_ngrams]

idx = np.arange(num_ngrams)


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def vectorise(texts, labels):
    kwargs = {
        'ngram_range' : (1, 2),
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : 'word',
        'min_df' : 2,
    }
    
    t_v = TfidfVectorizer(**kwargs)
    changed_text = t_v.fit_transform(texts)
    
       
    selector = SelectKBest(f_classif, k=min(20000, changed_text.shape[1]))
    selector.fit(changed_text, labels)
    changed_text = selector.transform(changed_text).astype('float32')
    return changed_text

v_data = vectorise(data['review'], data['sentiment'])

In [10]:
v_data.shape

(50000, 20000)

In [11]:
tr_texts = TfidfVectorizer().fit_transform(data['review'])
tr_texts.shape

(50000, 101895)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X = v_data.toarray()
y = (np.array(data['sentiment']))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
print("Train dataset shape: {0}, \nTest dataset shape: {1}".format(X_train.shape, X_test.shape))

Train dataset shape: (40000, 20000), 
Test dataset shape: (10000, 20000)


In [15]:
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout

In [16]:
model = keras.Sequential()

model = keras.Sequential()

model.add(Dropout(0.2, X_train.shape[1:]))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
    
model.add(Dense(1, activation='sigmoid'))



In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128)

Epoch 1/2
313/313 [==============================] - 13s 40ms/step - loss: 0.0755 - accuracy: 0.9718 - val_loss: 0.1961 - val_accuracy: 0.9287
Epoch 2/2
313/313 [==============================] - 12s 38ms/step - loss: 0.0678 - accuracy: 0.9752 - val_loss: 0.2063 - val_accuracy: 0.9287


In [22]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

313/313 [==============================] - 2s 5ms/step - loss: 0.2063 - accuracy: 0.9287
Test loss: 0.20634694397449493
Test accuracy: 0.9286999702453613


In [23]:
model.save_weights('model_weights.h5')
json_string = model.to_json()
f = open("model_architecture.json",'w')
f.write(json_string)
f.close()